In [1]:
from skimage import data as sd
import skimage.color as sc
from skimage.viewer import ImageViewer
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline  

pd.set_option('precision',10)

path = './Both/305776.jpg'

/Users/noel/anaconda/lib/python3.6/site-packages/skimage/viewer/utils/core.py:10: UserWarning: Recommended matplotlib backend is `Agg` for full skimage.viewer functionality.
  warn("Recommended matplotlib backend is `Agg` for full "


In [48]:
'''
opens image, converts to HSV, returns HSV array, height, width, total number of pixels
'''

def pre_process(path):
    img = sd.imread(path)
    img_hsv = sc.convert_colorspace(img, 'RGB', 'HSV')
    h = img_hsv.shape[0]
    w = img_hsv.shape[1]
    px = h * w
    
    return img_hsv, h, w, px

img_hsv, h, w, px = pre_process(path)

(600, 370)

In [3]:
'''
support function for features 1-3
'''

def make_dataframe(img_hsv):
    height = len(img_hsv)
    width = len(img_hsv[0])
    pixels = height * width
    hue = []
    saturation = []
    value = []
    
    for i in range(height):
        for j in range(width):
            hue.append(img_hsv[i][j][0])
            saturation.append(img_hsv[i][j][1])
            value.append(img_hsv[i][j][2])
            
    data = {'H': hue, 'S': saturation, 'V': value}
    hsv_df = pd.DataFrame(data=data)
    
    hue = np.array(hue)
    saturation = np.array(saturation)
    value = np.array(value)
    hue.shape = (h,w)
    saturation.shape = (h,w)
    value.shape = (h,w)
    
    return hsv_df, hue, saturation, value

hsv_df, hue, sat, val = make_dataframe(img_hsv)

In [4]:
'''
the first two features: average hue, average saturation
'''

def f1_2(hsv_df):
    return hsv_df.mean()['H'], hsv_df.mean()['S']

f1, f2 = f1_2(hsv_df)

In [5]:
'''
support function for features 3 to 23, k is number of bins
'''
def make_bins(hsv_df, k):
    bins = [0]
    increment = 1/k
    for i in range(k):
        temp = bins[i] + increment
        bins.append(temp)
    hue_bins = pd.cut(hsv_df['H'], bins=bins, include_lowest=True).value_counts()
    return hue_bins

hue_bins = make_bins(hsv_df, 20)

In [6]:
'''
count the num of hue bins that are most prominent, c is a multiplicatory component
'''
def f3(hue_bins, c):
    
    max_bins = max(hue_bins.tolist())
    count = 0
    
    for i in range(len(hue_bins)):
        if hue_bins[i] > c * max_bins:
            count +=1
            
    return count

f3 = f3(hue_bins, 0.1)

In [7]:
'''
features 4 to 23, the dispersion of each hue component, returns as list
'''

def f4_23(hue_bins, px):
    hue_bins = hue_bins.sort_index().tolist()
    color_dispersion = []
    for i in range(len(hue_bins)):
        color_dispersion.append(hue_bins[i]/px)
          
    return color_dispersion

f4_23 = f4_23(hue_bins, px)

In [8]:
from scipy import ndimage as ndi
import matplotlib.pyplot as plt

from skimage.morphology import watershed, disk
from skimage import data
from skimage.filters import rank
from skimage.util import img_as_ubyte

'''
performs water segmentation on images, which will be used for features
'''

def water_segmentation(path,hue,d1,d2,v,d3):

    image = img_as_ubyte(hue)
    image_original = sd.imread(path)
    image_gray = img_as_ubyte(sd.imread(path, as_grey=True))

    # denoise image
    denoised = rank.median(image, disk(d1))
    denoised_gray = rank.median(image_gray)

    # find continuous region (low gradient - where less than 10 for this image) --> markers
    # disk(5) is used here to get a more smooth image
    markers = rank.gradient(denoised, disk(d2)) < v
    markers = ndi.label(markers)[0]

    # local gradient (disk(2) is used to keep edges thin)
    gradient = rank.gradient(denoised, disk(d3))

    # process the watershed
    labels = watershed(gradient, markers)
    
    return image, image_original, denoised, markers, gradient, labels



In [9]:
'''
displayes the segmentation process
'''

def display_water_segmentation(image, image_original, denoised, markers, gradient, labels):

    # display results
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(8, 8), sharex=True, sharey=True, subplot_kw={'adjustable':'box-forced'})
    ax = axes.ravel()

    ax[0].imshow(image_original, cmap=plt.cm.gray, interpolation='nearest')
    ax[0].set_title("Original")

    ax[1].imshow(gradient, cmap=plt.cm.spectral, interpolation='nearest')
    ax[1].set_title("Local Gradient")

    ax[2].imshow(markers, cmap=plt.cm.spectral, interpolation='nearest')
    ax[2].set_title("Markers")

    ax[3].imshow(image_original, cmap=plt.cm.gray, interpolation='nearest')
    ax[3].imshow(labels, cmap=plt.cm.spectral, interpolation='nearest', alpha=.7)
    ax[3].set_title("Segmented")

    for a in ax:
        a.axis('off')

    fig.tight_layout()
    plt.show()
    
image, image_original, denoised, markers, gradient, labels = water_segmentation(path, hue, 3, 5, 10, 2) #original configuration for water shedding
# display_water_segmentation(image, image_original, denoised, markers, gradient, labels)

/Users/noel/anaconda/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [59]:
'''
support function to count the largest segments
'''

def make_segment_df(labels):
    labels_df = pd.DataFrame(data=labels)
    unique, counts = np.unique(labels, return_counts=True)

    data = {'seg': unique, 'px': counts}
    segment_df = pd.DataFrame(data=data)
    segment_df = segment_df.sort_values(by = 'px', ascending = False)
    
    return segment_df

segment_df = make_segment_df(labels)

seg_num = segment_df['seg'][1]
px = segment_df['px'][1]

def filter_labels(labels, n):
    arr = np.copy(labels)
    boole = arr[arr > n] = 0
    boole = arr[arr < n] = 0
    boole = arr[arr == n] = 1
    
    return arr

array = filter_labels(labels, seg_num)

from scipy import ndimage as nd

def center_of_mass(arr, h, w):
    coordinates = nd.measurements.center_of_mass(array)
    area = np.count_nonzero(arr)
    
    x_center = coordinates[0] / w
    y_center = coordinates[1] / h
    
    x = []
    y = []
    
    for i in range(h):
        for j in range(w):
            if arr[i][j] == 1:
                x.append(j)
                y.append(i)
                
    x = np.array(x)/w
    y = np.array(y)/h
    
    v1 = x.sum()/area ## in the original numbering this is f23-f25
    v2 = y.sum()/area ## f26-f28
    
    v3 = ( ((x - x_center) ** 2).sum() + ((y - y_center) ** 2).sum() ) / area ## f29-f31
    v4 = ( ((x - x_center) ** 3).sum() + ((y - y_center) ** 3).sum() ) / area ## f32-f34

    
    return v1, v2, v3, v4

f23, f24, f25, f26 = center_of_mass(array, h, w)

def f23_34(df, lab, height, width):
    f23_f25 = []
    f26_f28 = []
    f29_f31 = []
    f32_f34 = []
    
    for i in range(3):
        seg_num = segment_df['seg'][i]
        
        array = filter_labels(labels, seg_num)
        v1, v2, v3, v4 = center_of_mass(array, height, width)
        
        f23_f25.append(v1)
        f26_f28.append(v2)
        f29_f31.append(v3)
        f32_f34.append(v4)
        
    return f23_f25, f26_f28, f29_f31, f32_f34

f23_34(segment_df, labels, h, w)
        

([0.13386837915273095, 0.34731808731808733, 0.7707895897229069],
 [0.19334746261627223, 0.0044615384615384612, 0.17914969075077436],
 [0.042863197817754696, 0.1597326179505909, 0.6214339797090489],
 [0.0055544035282443036, 0.030184683109294463, 0.4907382618055075])

In [11]:
def show_gradient(gradient):

    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(20, 20), sharex=True, sharey=True, subplot_kw={'adjustable':'box-forced'})
    plt.imshow(gradient, cmap=plt.cm.spectral, interpolation='nearest')
    plt.axis('off')

    fig.tight_layout()
    plt.show()

In [12]:
nd.measurements.center_of_mass(img_hsv)

image = img_as_ubyte(sd.imread(path, as_grey=True))
print(image)
print(hue.shape)
image_2 = img_as_ubyte(hue)
print(image_2)

[[198 211 198 ..., 173 183 182]
 [206 219 208 ..., 193 185 184]
 [192 206 196 ..., 199 175 174]
 ..., 
 [100 145 116 ..., 149 144 130]
 [ 95 117  90 ..., 127 142 139]
 [122 100  56 ..., 115 137 134]]
(600, 370)
[[141 142 142 ..., 131 135 135]
 [141 142 142 ..., 131 135 135]
 [142 142 142 ..., 131 134 134]
 ..., 
 [ 14  14  13 ...,  17  18  18]
 [ 14  14  13 ...,  18  18  18]
 [ 14  13  12 ...,  18  19  19]]


/Users/noel/anaconda/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
